In [1]:
import numpy as np;
import pandas as pd;

## Initialize Parameters

In [2]:
## parameters and global variables

district = "tvm"

zetaPanchayat = 0.1
zetaMunicipality = 0.2
zetaCorporation = 0.3


muSmall = 0.09
muMedium = 0.04
muLarge = 0.02

## Load and Initialize Data

In [3]:
## Load population data
initDataDF = pd.read_csv("../data/" + district + "_init_data.csv")

## Load distance data
distanceData = pd.read_csv("../data/" + district + "_distance_matrix.csv")

## Load area data
areaData = pd.read_csv("../data/" + district + "_area_data.csv")

## Number of regions
r = len(initDataDF.index)

## Sort population data and distance data by name for easy matrix transformation
initDataDF.sort_values(by=['name'], inplace=True)
distanceData.sort_values(by=['fromName', 'toName'], inplace=True)
areaData.sort_values(by=['name'], inplace=True)

## TODO Change S to N 
## Rename mu as muFor

## Initialize Job 
initDataDF.loc[initDataDF['type']=='P','J'] = initDataDF[initDataDF['type']=='P']['S']*zetaPanchayat
initDataDF.loc[initDataDF['type']=='M','J'] = initDataDF[initDataDF['type']=='M']['S']*zetaMunicipality
initDataDF.loc[initDataDF['type']=='C','J'] = initDataDF[initDataDF['type']=='C']['S']*zetaCorporation

## Initialize T
areaData['areaSqKm'] = areaData['area']/(1000*1000)

initDataDF.loc[areaData['areaSqKm'] <= 25,'T'] = initDataDF[areaData['areaSqKm'] <= 25]['S']*muSmall
initDataDF.loc[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100),'T'] = initDataDF[(areaData['areaSqKm'] > 25) & (areaData['areaSqKm'] <= 100)]['S']*muMedium
initDataDF.loc[areaData['areaSqKm'] > 100,'T'] = initDataDF[areaData['areaSqKm'] > 100]['S']*muLarge

initDataDF.tail(15)

,name,type,S,E,I,H,R,J,T
62,Thirupuram__Thiruvananthapuram,P,18898,0,0,0,0,1889.8,1700.82
63,Thiruvananthapuram(C)__Thiruvananthapuram,C,966856,0,0,0,0,290056.8,19337.12
64,Tholikkodu__Thiruvananthapuram,P,25274,0,0,0,0,2527.4,1010.96
65,Uzhamalakkal__Thiruvananthapuram,P,21472,0,0,0,0,2147.2,1932.48
66,Vakkam__Thiruvananthapuram,P,16533,0,0,0,0,1653.3,1487.97
67,Vamanpuram__Thiruvananthapuram,P,21038,0,0,0,0,2103.8,1893.42
68,Varkala(M)__Thiruvananthapuram,M,40048,0,0,0,0,8009.6,3604.32
69,Vellanad__Thiruvananthapuram,P,31156,0,0,0,0,3115.6,2804.04
70,Vellarada__Thiruvananthapuram,P,40206,0,0,0,0,4020.6,1608.24
71,Vembayam__Thiruvananthapuram,P,38630,0,0,0,0,3863.0,1545.20


## Prepare Distance matrix

In [4]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
np.fill_diagonal(distanceMatrix,1)
distanceMatrix

array([[    1, 25049, 52074, ..., 17064, 19018, 19149],
       [25049,     1, 28368, ..., 12943, 16264, 14846],
       [52074, 28368,     1, ..., 35843, 36125, 42371],
       ...,
       [17064, 12943, 35843, ...,     1,  4233, 19475],
       [19018, 16264, 36125, ...,  4233,     1, 23689],
       [19149, 14846, 42371, ..., 19475, 23689,     1]])

In [5]:
initDataDF['J'].to_numpy()
np.tile(initDataDF['J'].to_numpy(),(r,1))

array([[1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       ...,
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9],
       [1592. , 3168.7, 1739.6, ..., 3621.2, 3176.1, 2624.9]])

In [6]:
a1 = np.asmatrix([[1,2], [3,4]])
a2 = np.asmatrix([[2,4], [2,4]])

np.multiply(a1,a1)
a1/a2

matrix([[0.5, 0.5],
        [1.5, 1. ]])

In [7]:
tPropMatrix = np.tile(initDataDF['J'].to_numpy(),(r,1))/np.multiply(distanceMatrix,distanceMatrix)
np.fill_diagonal(tPropMatrix,0)
tPropMatrix

array([[0.00000000e+00, 5.05010419e-06, 6.41516043e-07, ...,
        1.24362896e-05, 8.78141459e-06, 7.15847586e-06],
       [2.53724426e-06, 0.00000000e+00, 2.16168278e-06, ...,
        2.16163622e-05, 1.20071362e-05, 1.19095088e-05],
       [5.87085273e-07, 3.93752829e-06, 0.00000000e+00, ...,
        2.81866724e-06, 2.43376396e-06, 1.46209414e-06],
       ...,
       [5.46740667e-06, 1.89152124e-05, 1.35406869e-06, ...,
        0.00000000e+00, 1.77254647e-04, 6.92082440e-06],
       [4.40162842e-06, 1.19791607e-05, 1.33301086e-06, ...,
        2.02095189e-04, 0.00000000e+00, 4.67755935e-06],
       [4.34161056e-06, 1.43767993e-05, 9.68973659e-07, ...,
        9.54767394e-06, 5.65979514e-06, 0.00000000e+00]])

## Function Definitions

In [8]:
def getTij(i,j,distanceMatrix, initDataDF, tPropMatrix):
    Ti = initDataDF.iloc[i]['T']

    if(i==j):
        Ni = initDataDF.iloc[i]['S']
        return Ni - Ti; ## TODO Fix this later with Ni - Tij
    
    Tjprop = tPropMatrix[i][j]
    TjpropSum = tPropMatrix[i].sum();
    ## TODO move this tPropMatrix computation.
    return Ti*Tjprop/TjpropSum;
    
    

## Compute TMatrix

In [9]:
data = []

for i in range(0,r):
    for j in range(0,r):
        Tij = getTij(i,j,distanceMatrix, initDataDF, tPropMatrix)
        data.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij});
        
resultsDF = pd.DataFrame(data);


## Save output

In [10]:
resultsDF.to_csv("../output/" + district + "_tmatrix_results.csv")

In [ ]:
## TOP 5 per each region together with nos.